## STEP 1: Data Extraction from PostgresDB

In [1]:
# install scikit learn library
! pip install -U scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.1 MB 15.2 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1


In [6]:
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'msds610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'Harideep' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'cleaned' # schema we just created

In [4]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sqlalchemy import create_engine

# create a connection to the database
engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{db}')
connection = engine.connect()

In [8]:
# Pulling the data from the delivery_data table
import pandas as pd
query = 'SELECT * FROM raw.delivery_data;'
delivery_data = pd.read_sql_query(query, connection)
delivery_data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Unnamed: 14,Traffic_Level,Distance (km),TARGET
0,4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,17.11,77.0,0.0,haze,None,Low,NaN,21.66666667
1,B379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,19.50,93.0,0.0,mist,None,Very High,37.17,85.26666667
2,5D6D,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,20.45,91.0,0.0,mist,None,Low,3.34,28.58333333
3,7A6A,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,23.86,78.0,0.0,mist,None,Moderate,10.05,35.18333333
4,70A2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,26.55,87.0,0.0,mist,None,High,9.89,43.45


## STEP 2: Data Cleaning [Transformations]

In [13]:
delivery_data.shape

(9035, 17)

In [14]:
# Checkiing for missing values
delivery_data.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Type_of_order                  0
Type_of_vehicle                0
temperature                    0
humidity                       0
precipitation                  0
weather_description            0
Traffic_Level                  0
Distance (km)                  0
TARGET                         0
dtype: int64

In [15]:
# Checking for duplicates
delivery_data.duplicated().sum()

0

In [16]:
delivery_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9035 entries, 1 to 9999
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           9035 non-null   object 
 1   Delivery_person_ID           9035 non-null   object 
 2   Delivery_person_Age          9035 non-null   float64
 3   Delivery_person_Ratings      9035 non-null   float64
 4   Restaurant_latitude          9035 non-null   float64
 5   Restaurant_longitude         9035 non-null   float64
 6   Delivery_location_latitude   9035 non-null   float64
 7   Delivery_location_longitude  9035 non-null   float64
 8   Type_of_order                9035 non-null   object 
 9   Type_of_vehicle              9035 non-null   object 
 10  temperature                  9035 non-null   float64
 11  humidity                     9035 non-null   float64
 12  precipitation                9035 non-null   float64
 13  weather_description    

In [17]:
delivery_data.Traffic_Level.value_counts()

Traffic_Level
High         2727
Moderate     2212
Very High    1747
Low          1717
Very Low      632
Name: count, dtype: int64

In [18]:
delivery_data.weather_description.value_counts()

weather_description
clear sky           3210
haze                2376
mist                1739
broken clouds        532
smoke                497
scattered clouds     409
overcast clouds      182
fog                   49
few clouds            40
moderate rain          1
Name: count, dtype: int64

In [19]:
delivery_data.Type_of_order.value_counts()

Type_of_order
Snack      2309
Meal       2279
Drinks     2274
Buffet     2173
Name: count, dtype: int64

In [20]:
delivery_data.Type_of_vehicle.value_counts()

Type_of_vehicle
motorcycle           5345
scooter              2971
electric_scooter      709
bicycle                10
Name: count, dtype: int64

In [21]:
# Catagozing the traffic level to numerical values
traffic_level = {'Very High': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very Low': 0}
weather_description = delivery_data.weather_description.unique()
Type_of_order = delivery_data.Type_of_order.unique()
Type_of_vehicle = delivery_data.Type_of_vehicle.unique()

# Create a dictionary for the weather_description, Type_of_order and Type_of_vehicle with numerical values
weather_description_dict = {weather_description[i]: i for i in range(len(weather_description))}
Type_of_order_dict = {Type_of_order[i]: i for i in range(len(Type_of_order))}
Type_of_vehicle_dict = {Type_of_vehicle[i]: i for i in range(len(Type_of_vehicle))}

In [22]:
# Converting the weather_description, Type_of_order and Type_of_vehicle to numerical values
delivery_data['Traffic_Level'] = delivery_data['Traffic_Level'].map(traffic_level)
delivery_data['weather_description'] = delivery_data['weather_description'].map(weather_description_dict)
delivery_data['Type_of_order'] = delivery_data['Type_of_order'].map(Type_of_order_dict)
delivery_data['Type_of_vehicle'] = delivery_data['Type_of_vehicle'].map(Type_of_vehicle_dict)

In [23]:
delivery_data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Traffic_Level,Distance (km),TARGET
1,B379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,0,0,19.50,93.0,0.0,0,4,37.17,85.26666667
2,5D6D,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,1,1,20.45,91.0,0.0,0,1,3.34,28.58333333
3,7A6A,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2,1,23.86,78.0,0.0,0,2,10.05,35.18333333
4,70A2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,0,0,26.55,87.0,0.0,0,3,9.89,43.45
5,9BB4,HYDRES09DEL03,22.0,4.8,17.431668,78.408321,17.461668,78.438321,2,1,21.43,65.0,0.0,1,2,11.30,30.6


In [24]:
# Parameter Extraction Haversine Distance: The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):

    # Radius of the Earth in km
    R = 6371.0

    # Converting latitude and longitude from degrees to radians
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Calculating the change in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2

    return a

**Parameter Extraction Haversine Distance: The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.**

In [25]:
delivery_data['haversine_distance'] = haversine_distance(delivery_data['Restaurant_latitude'], 
                                                         delivery_data['Restaurant_longitude'], 
                                                         delivery_data['Delivery_location_latitude'], 
                                                         delivery_data['Delivery_location_longitude'])

In [26]:
# Dropping the columns that are not needed
delivery_data.drop(columns=['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude'], inplace=True)

In [27]:
delivery_data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Traffic_Level,Distance (km),TARGET,haversine_distance
1,B379,BANGRES18DEL02,34.0,4.5,0,0,19.50,93.0,0.0,0,4,37.17,85.26666667,2.509104e-06
2,5D6D,BANGRES19DEL01,23.0,4.4,1,1,20.45,91.0,0.0,0,1,3.34,28.58333333,1.485020e-08
3,7A6A,COIMBRES13DEL02,38.0,4.7,2,1,23.86,78.0,0.0,0,2,10.05,35.18333333,3.738044e-07
4,70A2,CHENRES12DEL01,32.0,4.6,0,0,26.55,87.0,0.0,0,3,9.89,43.45,2.375348e-07
5,9BB4,HYDRES09DEL03,22.0,4.8,2,1,21.43,65.0,0.0,1,2,11.30,30.6,1.309168e-07


In [28]:
# Data scaling using standard scaler
from sklearn.preprocessing import StandardScaler

numeric_cols = ['Distance (km)', 'haversine_distance', 'Delivery_person_Age', 'Delivery_person_Ratings', 'temperature', 'humidity', 'precipitation', 'precipitation', 'TARGET']

scaler = StandardScaler()
delivery_data[numeric_cols] = scaler.fit_transform(delivery_data[numeric_cols])
delivery_data = delivery_data.round(2)

delivery_data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Traffic_Level,Distance (km),TARGET,haversine_distance
1,B379,BANGRES18DEL02,0.79,-0.39,0,0,-0.93,1.85,-0.02,0,4,2.73,2.88,2.30
2,5D6D,BANGRES19DEL01,-1.14,-0.70,1,1,-0.65,1.72,-0.02,0,1,-1.31,-0.55,-1.01
3,7A6A,COIMBRES13DEL02,1.49,0.23,2,1,0.37,0.87,-0.02,0,2,-0.50,-0.15,-0.53
4,70A2,CHENRES12DEL01,0.44,-0.08,0,0,1.17,1.46,-0.02,0,3,-0.52,0.35,-0.71
5,9BB4,HYDRES09DEL03,-1.32,0.54,2,1,-0.36,0.03,-0.02,1,2,-0.36,-0.43,-0.85


## Step 3: Loading

In [29]:
# Inserting the data into the database as star schema
delivery_data.to_sql('delivery_data', engine, schema='cleaned', if_exists='replace', index=False, method='multi', chunksize=1000)


# creating dimension tables from above dictionaries
weather_description_df = pd.DataFrame(weather_description_dict.items(), columns=['weather_description', 'weather_description_id'])
Type_of_order_df = pd.DataFrame(Type_of_order_dict.items(), columns=['Type_of_order', 'Type_of_order_id'])
Type_of_vehicle_df = pd.DataFrame(Type_of_vehicle_dict.items(), columns=['Type_of_vehicle', 'Type_of_vehicle_id'])
traffic_level_df = pd.DataFrame(traffic_level.items(), columns=['Traffic_Level', 'Traffic_Level_id'])


# Inserting the data into the database as star schema
weather_description_df.to_sql('weather_description', engine, schema='cleaned', if_exists='replace', index=False)
Type_of_order_df.to_sql('Type_of_order', engine, schema='cleaned', if_exists='replace', index=False)
Type_of_vehicle_df.to_sql('Type_of_vehicle', engine, schema='cleaned', if_exists='replace')
traffic_level_df.to_sql('traffic_level', engine, schema='cleaned', if_exists='replace')


5

In [30]:
# printing the tables in the cleaned schema
query = 'SELECT table_name FROM information_schema.tables WHERE table_schema=\'cleaned\';'
table_names = pd.read_sql_query(query, engine)
table_names

,table_name
0,delivery_data
1,weather_description
2,Type_of_order
3,Type_of_vehicle
4,traffic_level
